## Example: HXMM01 
### Prepare visibiity data from the calibrated MS restored by the local ALMA pipeline

#### Load some essential Python modules + CASA 6 tasks/tools 

In [1]:

import sys,os,glob,io,socket
import logging
from pprint import pprint

import casatasks as ctasks
import casatools as ctools

# Import wurlitzer for display real-time console logs
#   https://github.com/minrk/wurlitzer
%reload_ext wurlitzer

# for inline plots
%matplotlib inline
%config InlineBackend.figure_format = "retina"


Using the beta-version of CASA 6

In [2]:
print('casatools ver:',ctools.version_string())
print('casatasks ver:',ctasks.version_string())

casatools ver: 2019.166
casatasks ver: 2019.162


#### Create some convinient functions

In [3]:
def rmPointing(outvis):
    """
    Empty the POINTING table
    """
    os.system("du -hs "+outvis)
    ctb=ctools.table()
    ctb.open(outvis+'/POINTING',nomodify=False)
    ctb.removerows(range(ctb.nrows()))
    ctb.done()
    # rmtables(outvis+'/POINTING') # doesn't work
    os.system("du -hs "+outvis)
    
def setLogfile(logfile,overwrite=False,rmlast=False,onconsole=True):
    """
    Set CASA log file
    """
    if  overwrite==True:
        os.system('rm -rf '+logfile)
    if  rmlast==True:
        os.system('rm -rf '+ctasks.casalog.logfile())
    ctasks.casalog.setlogfile(logfile)
    ctasks.casalog.showconsole(onconsole=onconsole)

### 2015.1.00723.S (Band 6, Cycle-3, Xue et al. 2018) 

#### Spws Setup

+ spw=0,4,8,12    228GHz   H2O 211-202
+ spw=1,5,9,13    231GHz   Continuum
+ spw=2,6,10,14   243GHz   CI 2-1
+ spw=3,7,11,15   242GHz   CO76/CI 7-6

#### Export one MS per baseband

+ Combine the SPWs from each baseband into a single SPW
+ Transform the frame from TOPO to LSRK
+ Exclude the edge channels

Note: GMaKE requires the input MS contains only one SPW and one FIELD.

#### MS manupilation

+ The integration time is rebined to 60s (limited by the time-averging smearing)
+ The continuum SPW is averged into 1 representive channel (limited by the bandwidth smearing)
+ The point tables were removed[<sup>1</sup>](#fn1): they are only used for mosaic imaging, not in this specific case; for ALMA data, the table can be quite large even for channle-/time-averaged data)

<span id="fn1"> <sup>1</sup></span> We can't remove ms/POINTING table using `rmtables()` as its abence will crash certain CASA tasks/tools (e.g. listable); instead, we set it to have empty row, as it's done in `rmPointing()`.

In [4]:

# Switch working directory

demo_dir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/hxmm01/alma/2015.1.00723.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_hxmm01.log')

vis_name='/Volumes/D1/projects/hxmm01/proc/2015.1.00723.S/calibrated/calibrated.ms.split.cal'
#ctasks.listobs(vis_name)

spw_list=['0,4,8,12','1,5,9,13','2,6,10,14','3,7,11,15']
bb_list=['1','2','3','4']
field='HXMM01'

# For BB1 & BB2: TOPO->LSKR

for i in range(0,2):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='HXMM01',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=9,nchan=110,width=1,
                        timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)


# For BB3+BB4: TOPO->LSKR

outvis='bb34.ms'
os.system('rm -rf '+outvis)
ctasks.mstransform(vis_name,outputvis=outvis,field='HXMM01',spw='3,2,7,6,11,10,15,14',datacolumn='data',scan='',
                     regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=10,nchan=161,width=1,
                     timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                     keepflags=False,usewtspectrum=False)         
rmPointing(outvis)

# For BB2: Channel-averging

for i in range(1,2):
    outvis='bb'+bb_list[i]+'.mfs.ms'
    os.system('rm -rf '+'bb'+bb_list[i]+'.mfs.ms')
    ctasks.mstransform('bb'+bb_list[i]+'.ms',outputvis=outvis,field='HXMM01',datacolumn='data',
                        chanaverage=True,chanbin=110,
                        timeaverage=False,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)
   

2019-10-11 18:29:37	INFO	mstransform::::casa	##########################################
2019-10-11 18:29:37	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:29:37	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2015.1.00723.S/calibrated/calibrated.ms.split.cal', outputvis='bb1.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='HXMM01', spw='0,4,8,12', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=110, start=9, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorde

2.4G	bb1.ms
250M	bb1.ms


2019-10-11 18:33:00	INFO	mstransform::::casa	##########################################
2019-10-11 18:33:00	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:33:00	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2015.1.00723.S/calibrated/calibrated.ms.split.cal', outputvis='bb2.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='HXMM01', spw='1,5,9,13', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=110, start=9, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorde

2.4G	bb2.ms
234M	bb2.ms


2019-10-11 18:36:29	INFO	mstransform::::casa	##########################################
2019-10-11 18:36:29	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:36:29	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2015.1.00723.S/calibrated/calibrated.ms.split.cal', outputvis='bb34.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='HXMM01', spw='3,2,7,6,11,10,15,14', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=161, start=10, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitsp

2.5G	bb34.ms
350M	bb34.ms


2019-10-11 18:41:59	INFO	mstransform::::casa	##########################################
2019-10-11 18:41:59	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:41:59	INFO	mstransform::::casa	mstransform( vis='bb2.ms', outputvis='bb2.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='HXMM01', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=110, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fal

 39M	bb2.mfs.ms
 39M	bb2.mfs.ms


2019-10-11 18:42:04	INFO	mstransform::::casa	Result mstransform: True
2019-10-11 18:42:04	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-10-11 13:41:59.079658 End time: 2019-10-11 13:42:04.359815
2019-10-11 18:42:04	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-10-11 18:42:04	INFO	mstransform::::casa	##########################################



### 2011.0.00539.S (Band 7, Cycle-0)

#### Spws Setup

+ spw=0  336GHz   H2O 111-000
+ spw=1  338GHz   Continuum
+ spw=2  348GHz   CO10-9 / H2O312-221
+ spw=3  350GHz   Continuum


In [5]:
# Switch working directory

demo_dir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/hxmm01/alma/2011.0.00539.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_hxmm01.log')

vis_name='/Volumes/D1/projects/hxmm01/proc/2011.0.00539.S/calibrated/calibrated.ms.split.cal'
#ctasks.listobs(vis_name)

spw_list=['0','1','2','3']
bb_list=['1','2','3','4']
field='XMM01'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR

for i in range(0,4):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='XMM01',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=8,nchan=112,width=1,
                        timeaverage=False,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)

# For BB2/BB4: Channel-averging

for i in range(0,4):
    if  bb_list[i]=='1' or bb_list[i]=='3':
        continue
    outvis='bb'+bb_list[i]+'.mfs.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform('bb'+bb_list[i]+'.ms',outputvis=outvis,field='XMM01',datacolumn='data',
                        chanaverage=True,chanbin=112,
                        timeaverage=False,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)


2019-10-11 18:42:05	INFO	mstransform::::casa	##########################################
2019-10-11 18:42:05	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:42:05	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2011.0.00539.S/calibrated/calibrated.ms.split.cal', outputvis='bb1.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='XMM01', spw='0', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=112, start=8, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=False, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, wa

 44M	bb1.ms
 44M	bb1.ms


2019-10-11 18:42:09	INFO	mstransform::::casa	##########################################
2019-10-11 18:42:09	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:42:09	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2011.0.00539.S/calibrated/calibrated.ms.split.cal', outputvis='bb2.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='XMM01', spw='1', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=112, start=8, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=False, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, wa

 44M	bb2.ms
 44M	bb2.ms


2019-10-11 18:42:13	INFO	mstransform::::casa	##########################################
2019-10-11 18:42:13	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:42:13	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hxmm01/proc/2011.0.00539.S/calibrated/calibrated.ms.split.cal', outputvis='bb3.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='XMM01', spw='2', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=112, start=8, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=False, timebin='60s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, wa

...

2019-10-11 18:42:20	INFO	mstransform::::casa	##########################################
2019-10-11 18:42:20	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:42:20	INFO	mstransform::::casa	mstransform( vis='bb2.ms', outputvis='bb2.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='XMM01', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=112, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

7.1M	bb2.mfs.ms
7.1M	bb2.mfs.ms


2019-10-11 18:42:23	INFO	mstransform::::casa	##########################################
2019-10-11 18:42:23	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-11 18:42:23	INFO	mstransform::::casa	mstransform( vis='bb4.ms', outputvis='bb4.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='XMM01', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=112, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

7.1M	bb4.mfs.ms
7.1M	bb4.mfs.ms


2019-10-11 18:42:25	INFO	mstransform::::casa	Result mstransform: True
2019-10-11 18:42:25	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-10-11 13:42:22.694760 End time: 2019-10-11 13:42:25.022492
2019-10-11 18:42:25	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-10-11 18:42:25	INFO	mstransform::::casa	##########################################
